In [34]:
import sys 
from FR3Py.robot.interface import FR3Real
robot = FR3Real(robot_id='fr3')

Interface Running...


In [35]:
robot.getStates()

{'q': array([ 7.57132296e-04, -7.84388364e-01,  3.87324241e-03, -2.40504641e+00,
        -1.38597921e-03,  1.57104123e+00,  7.86667680e-01]),
 'dq': array([-0.00116993, -0.0002909 , -0.00040437, -0.00012203,  0.00020795,
         0.00017607,  0.00054925]),
 'T': array([-0.06021428, -4.34961081, -0.9263519 , 23.3774147 ,  1.08232343,
         2.09470344, -0.10297176])}

In [36]:
import numpy as np
from FR3Py.robot.model_collision_avoidance import PinocchioModel
import time

pin_robot = PinocchioModel()
joint_lb = np.array([-2.3093, -1.5133, -2.4937, -2.7478, -2.48, 0.8521, -2.6895])
joint_ub = np.array([2.3093, 1.5133, 2.4937, -0.4461, 2.48, 4.2094, 2.6895])
n_joints = 7
q_bar = 0.5*(joint_ub + joint_lb)
P_EE_desired = np.array([0.42, 0.50, 0.0])
R_EE_desired = np.array([[1, 0, 0],
                        [0, -1, 0],
                        [0, 0, -1]])
P_EE_initial = np.array([0.30, 0.0, 0.47])

### Inverse dynamic control (initial pose to pre-cleaning pose)

In [37]:
from cores.utils.trajectory_utils import TrapezoidalTrajectory

via_points = np.array([P_EE_initial, P_EE_desired])
target_time = np.array([0, 10])
traj = TrapezoidalTrajectory(via_points, target_time, T_antp=0.2, Ts=0.01)

In [38]:
from cores.utils.control_utils import get_torque_to_track_traj_const_ori

T = 12

Kp_task = np.diag([40,40,40,100,100,100])
Kd_task = np.diag([40,40,40,100,100,100])

Kp_joint = np.diag([0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1])*10
Kd_joint = np.diag([0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5])*10

delta = 0.1

t_start = time.time()
while time.time() - t_start < T:
    t = time.time() - t_start
    traj_pos, traj_vel, traj_acc = traj.get_traj_and_ders(t)

    q = 0.025*np.ones(9)
    dq = np.zeros(9)
    robot_info = robot.getStates()
    q[0:n_joints] = robot_info['q']
    dq[0:n_joints] = robot_info['dq']

    pin_info = pin_robot.getInfo(q, dq)
    q = q[0:n_joints]
    dq = dq[0:n_joints]
    P_EE = pin_info["P_EE"]
    R_EE = pin_info["R_EE"]
    J_EE = pin_info["J_EE"][:,0:n_joints] # shape (6,7)
    dJdq_EE = pin_info["dJdq_EE"] # shape (6,)
    v_EE = J_EE @ dq # shape (6,)
    M = pin_info["M"][0:n_joints,0:n_joints] + delta*np.eye(n_joints) # shape (7,7)
    # Minv = pin_info["Minv"][0:n_joints,0:n_joints]   # shape (7,7)
    Minv = np.linalg.inv(M)
    nle = pin_info["nle"][0:n_joints]  # shape (7,)
    G = pin_info["G"][0:n_joints]  # shape (7,)

    S, u_task = get_torque_to_track_traj_const_ori(traj_pos, traj_vel, traj_acc, R_EE_desired, Kp_task, Kd_task, Minv, J_EE, dJdq_EE, dq, P_EE, R_EE)

    # Secondary objective: encourage the joints to remain close to the initial configuration
    W = np.diag(1.0/(joint_ub-joint_lb))
    eq = W @ (q - q_bar)
    deq = W @ dq
    u_joint = M @ (- Kd_joint @ deq - Kp_joint @ eq) 

    # Compute the input torque
    Spinv = S.T @ np.linalg.pinv(S @ S.T + 0.01* np.eye(S.shape[0]))
    u_nominal =  Spinv @ u_task + (np.eye(len(q)) - Spinv @ S) @ u_joint
    u = np.clip(u_nominal, -20, 20)
    robot.setCommands(u)

robot.setCommands(np.zeros_like(u))


### (Optional) Press against the surface while maintaining only orientation

In [21]:
from cores.utils.control_utils import get_torque_to_track_const_ori

T = 1000

Kp_task = np.diag([100,100,100])
Kd_task = np.diag([100,100,100])

Kd_joint = np.diag([0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5])*10
F_ext = np.array([0, 0, -15, 0, 0, 0])

delta = 0.1

t_start = time.time()
while time.time() - t_start < T:
    t = time.time() - t_start
    
    q = 0.025*np.ones(9)
    dq = np.zeros(9)
    robot_info = robot.getStates()
    q[0:n_joints] = robot_info['q']
    dq[0:n_joints] = robot_info['dq']

    pin_info = pin_robot.getInfo(q, dq)
    q = q[0:n_joints]
    dq = dq[0:n_joints]
    P_EE = pin_info["P_EE"]
    R_EE = pin_info["R_EE"]
    J_EE = pin_info["J_EE"][:,0:n_joints] # shape (6,7)
    dJdq_EE = pin_info["dJdq_EE"] # shape (6,)
    v_EE = J_EE @ dq # shape (6,)
    M = pin_info["M"][0:n_joints,0:n_joints] + delta*np.eye(n_joints) # shape (7,7)
    # Minv = pin_info["Minv"][0:n_joints,0:n_joints]   # shape (7,7)
    Minv = np.linalg.inv(M)
    nle = pin_info["nle"][0:n_joints]  # shape (7,)
    G = pin_info["G"][0:n_joints]  # shape (7,)

    S, u_task = get_torque_to_track_const_ori(R_EE_desired, Kp_task, Kd_task, Minv, J_EE, dJdq_EE, dq, R_EE)

    # Second objective: apply a force on the z axis to press the end-effector against the table
    u_press = J_EE.T @ F_ext

    # Third objective: encourage the joints to have zero velocity
    W = np.diag(1.0/(joint_ub-joint_lb))
    deq = W @ dq
    u_joint = M @ (-Kd_joint @ deq) 

    # Compute the input torque
    Spinv = S.T @ np.linalg.pinv(S @ S.T + 0.01* np.eye(S.shape[0]))
    u_nominal =  Spinv @ u_task + (np.eye(len(q)) - Spinv @ S) @ u_joint + u_press
    u = np.clip(u_nominal, -20, 20)
    robot.setCommands(u)

robot.setCommands(np.zeros_like(u))

KeyboardInterrupt: 

### Press against the surface while tracking a circular trajectory

In [39]:
from cores.utils.trajectory_utils import CircularTrajectory
duration = 100
P_start_point = P_EE_desired # np.array([0.42, 0.50, 0.00])
P_center = np.array([0.30, 0.50, 0.00])
nominal_linear_vel = 0.1
R_b_to_w = np.eye(3)
traj = CircularTrajectory(P_center, P_start_point, nominal_linear_vel, R_b_to_w, duration)


In [45]:
T = 60

Kp_task = np.diag([40,40,40,100,100,100])
Kd_task = np.diag([40,40,40,100,100,100])

Kp_joint = np.diag([0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1])*200
Kd_joint = np.diag([0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5])*160

F_ext = np.array([0, 0, -10, 0, 0, 0])

delta = 0.1

t_start = time.time()
while time.time() - t_start < T:
    t = time.time() - t_start
    traj_pos, traj_vel, traj_acc = traj.get_traj_and_ders(t)

    q = 0.025*np.ones(9)
    dq = np.zeros(9)
    robot_info = robot.getStates()
    q[0:n_joints] = robot_info['q']
    dq[0:n_joints] = robot_info['dq']

    pin_info = pin_robot.getInfo(q, dq)
    q = q[0:n_joints]
    dq = dq[0:n_joints]
    P_EE = pin_info["P_EE"]
    R_EE = pin_info["R_EE"]
    J_EE = pin_info["J_EE"][:,0:n_joints] # shape (6,7)
    dJdq_EE = pin_info["dJdq_EE"] # shape (6,)
    v_EE = J_EE @ dq # shape (6,)
    M = pin_info["M"][0:n_joints,0:n_joints] + delta*np.eye(n_joints) # shape (7,7)
    # Minv = pin_info["Minv"][0:n_joints,0:n_joints]   # shape (7,7)
    Minv = np.linalg.inv(M)
    nle = pin_info["nle"][0:n_joints]  # shape (7,)
    G = pin_info["G"][0:n_joints]  # shape (7,)

    S, u_task = get_torque_to_track_traj_const_ori(traj_pos, traj_vel, traj_acc, R_EE_desired, Kp_task, Kd_task, Minv, J_EE, dJdq_EE, dq, P_EE, R_EE)

    # Second objective: apply a force on the z axis to press the end-effector against the table
    u_press = J_EE.T @ F_ext

    # Third objective: encourage the joints to remain close to the initial configuration
    W = np.diag(1.0/(joint_ub-joint_lb))
    eq = W @ (q - q_bar)
    deq = W @ dq
    u_joint = M @ (- Kd_joint @ deq - Kp_joint @ eq) 

    # Compute the input torque
    Spinv = S.T @ np.linalg.pinv(S @ S.T + 0.01* np.eye(S.shape[0]))
    u_nominal =  Spinv @ u_task + (np.eye(len(q)) - Spinv @ S) @ u_joint + u_press
    u = np.clip(u_nominal, -20, 20)
    robot.setCommands(u)

robot.setCommands(np.zeros_like(u))